# Capturing neutral face video

# Rendering face
## Remember to only run this once the headset is on! 

In [ ]:
import cv2 as cv
import numpy as np
import pickle
from matplotlib import pyplot as plt

import face_alignment
import time
import cv2
import scipy.optimize as opt
from calibration.undistort import undistort

def draw_marks(image, marks, color=(0, 255, 0)):
    """
    Draw the facial landmarks on an image
    Parameters
    ----------
    image : np.uint8
        Image on which landmarks are to be drawn.
    marks : list or numpy array
        Facial landmark points
    color : tuple, optional
        Color to which landmarks are to be drawn with. The default is (0, 255, 0).
    Returns
    -------
    None.
    """
    for mark in marks:
        cv.circle(image, (mark[0], mark[1]), 2, color, -1, cv.LINE_AA)
        

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cuda')

# this function compares the differences between two sets of points
# each column is a point
def err(p1, p2):
    # hopefully this is actually correct
    return sum(sum((p1 - p2)*(p1 - p2)))


# x is an array of 9 elements
def to_optimize(x, original_points, target_points):
    # reshape it to be a 3x3 transformation matrix
    transform = x.reshape((3,3))
    temp = transform@original_points
    result = np.zeros((2,original_points.shape[1]))
    for i in range(original_points.shape[1]):
        result[0,i] = temp[0,i]/temp[2,i]
        result[1,i] = temp[1,i]/temp[2,i]
    return err(result, target_points[:2,:])

def crop_image(image):
    return image[:,80:-80,:] ## 480x640x3 -> 480x480x3

def capture_initial_video():
    original_points = 0
    target_points = 0
    cap = cv2.VideoCapture(0)
    
    ## capture jawline with headset on and save the feature pts for alignment 
    while True:
        res, frame = cap.read()
        if not res:
            print('no frame')
            continue
        cv2.imshow(winname="RAW FRAME", mat=frame)

#         frame = undistort(frame)
#         frame = frame[:,80:-80,:]
        frame = crop_image(frame)
        frame = cv.resize(frame,(256,256))
        
        preds = fa.get_landmarks(frame)
        if(preds):
            # only take the jawline and the mouth keypoints 
            headset_features = np.concatenate((preds[0].astype(int)[:17],preds[0].astype(int)[49:61]))
            draw_marks(frame,headset_features,color = (255,0,0))
        cv2.imshow(winname="Face", mat=cv2.resize(frame,(720,720)))
        if cv2.waitKey(5) & 0xFF == ord('q'):
            target_points = np.concatenate((preds[0].astype(int)[:17],preds[0].astype(int)[49:61]))
            target_points = np.append(target_points, np.ones((target_points.shape[0],1)), axis=1)
            target_points = target_points.T
            print("got target points with shape", target_points.shape)
            break   
        
    video_out = cv2.VideoWriter('./init_video.mp4',cv2.VideoWriter_fourcc(*'XVID'),25,(256,256))
    print('now trying to find a good no-headset match')
    record = False
    
    ## align face without headset using the saved jawline and mouth keypoints 
    result_marks = 0
    generated_result = False
    while True:
        res, frame = cap.read()
        if not res:
            print('no frame')
            continue
        
        frame = crop_image(frame)
        frame = cv.resize(frame,(256,256))
        
        # show the image
        preds = fa.get_landmarks(frame)
        if(preds):
            new_face = preds[0].astype(int)
            clean_frame = frame.copy()

            draw_marks(frame,new_face,color = (0,0,255))
        if generated_result:
            print("generating result")
            draw_marks(frame,result_marks, color = (0,255,0))
        draw_marks(frame,headset_features,color = (255,0,0))
        cv2.imshow(winname="Face", mat=cv2.resize(frame,(720,720)))

        pressed_key = cv2.waitKey(5) & 0xFF

        if(pressed_key == ord('q')):
            print("stop recording")
            break
        elif(pressed_key == ord('r')):
            print('started recording')
            record = True
            original_points = np.concatenate((preds[0].astype(int)[:17],preds[0].astype(int)[49:61]))
            original_points = np.append(original_points, np.ones((original_points.shape[0],1)), axis=1)
            original_points = original_points.T
            print(original_points.shape)
            res = opt.minimize(to_optimize, np.array([1,0,0,0,1,0,0,0,1]), args=(original_points, target_points))
            transform = res.x.reshape((3,3))
            temp = transform@original_points
            result = np.zeros((2,original_points.shape[1]))
            for i in range(original_points.shape[1]):
                result[0,i] = temp[0,i]/temp[2,i]
                result[1,i] = temp[1,i]/temp[2,i]
            result_marks = result.T.astype(int)
            generated_result = True
            print(result_marks)
        if(record):
            video_out.write(clean_frame)
    # When everything done, release the video capture and video write objects
    cap.release()
    video_out.release()
    # Close all windows
    cv2.destroyAllWindows()
    pickle.dump(clean_frame,open('./starting_picture.p','wb'))
    return clean_frame

print("run")

capture_initial_video()
cv2.destroyAllWindows()

# Attention: 
## Run this before the next cell:
### https://stackoverflow.com/questions/70775129/runtimeerror-v4l2loopback-backend-stdexception-when-using-pyvirtualcam


!sudo modprobe -r v4l2loopback && sudo modprobe v4l2loopback devices=1 video_nr=4 card_label="Virtual" exclusive_caps=1 max_buffers=2


In [ ]:
#### import matplotlib
# matplotlib.use('Agg')
import os, sys
import yaml
from argparse import ArgumentParser
from tqdm import tqdm

import imageio
import numpy as np
from skimage.transform import resize
from skimage import img_as_ubyte
import torch
from sync_batchnorm import DataParallelWithCallback

from modules.generator import OcclusionAwareGenerator
from modules.keypoint_detector import KPDetector
from animate import normalize_kp
from scipy.spatial import ConvexHull
import pdb
#import pyvirtualcam
import time
from calibration.undistort import undistort
import cv2


def crop_image(image):
    return image[:,80:-80,:]

if sys.version_info[0] < 3:
    raise Exception("You must use Python 3 or higher. Recommended version is Python 3.7")



reader2 = cv2.VideoCapture('./init_video.mp4')
regular_video = []

def pre_process_frame(frame):
    #frame = frame[:,80:-80,:]
    frame = crop_image(frame)
    frame = cv2.resize(frame,(256,256))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
    frame = [resize(img, (256, 256))[..., :3] for img in [frame]]
    return frame

        
while(1):
    ret,frame = reader2.read()
    if(ret):
        regular_video.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    else:
        break

regular_video = [resize(frame, (256, 256))[..., :3] for frame in regular_video]
cap = cv2.VideoCapture(0)
res, frame = cap.read()
while(res is not True):
    res, frame = cap.read()
    print('retry')
    time.sleep(0.3)

height_black_band = [105, 120]
lateral_shift = 2
left_right_black_size = 90
bottom_black_size = 70
counter = 1


while(counter < 30000):
        start = time.time()
        res, frame = cap.read()
        if not res:
            print('no frame')

        frame = pre_process_frame(frame) 
        frame[0][:height_black_band[0],:] = regular_video[counter%len(regular_video)][:height_black_band[0],:] #256 x 256
        
        frame[0][height_black_band[0]:height_black_band[1],:] = np.zeros((height_black_band[1] - height_black_band[0], 256, 3))

        ## shift chin 
        frame[0][height_black_band[1]:,left_right_black_size + lateral_shift:256 - left_right_black_size + lateral_shift,:] = \
            frame[0][height_black_band[1]:,left_right_black_size :256 - left_right_black_size,:]
        
        frame[0][height_black_band[1]:256,0:left_right_black_size] = \
            np.zeros((256 - height_black_band[1], left_right_black_size, 3))
        frame[0][height_black_band[1]:256,-left_right_black_size:] = \
            np.zeros((256 - height_black_band[1], left_right_black_size, 3))
        frame[0][-bottom_black_size:,:] = np.zeros((bottom_black_size, 256,3))

#         frame[0][height_black_band[0]:height_black_band[1],:] = np.ones((height_black_band[1] - height_black_band[0], 256, 3))*0.5

#         ## shift chin 
#         frame[0][height_black_band[1]:,left_right_black_size + lateral_shift:256 - left_right_black_size + lateral_shift,:] = \
#             frame[0][height_black_band[1]:,left_right_black_size :256 - left_right_black_size,:]
        
#         frame[0][height_black_band[1]:256,0:left_right_black_size] = \
#             np.ones((256 - height_black_band[1], left_right_black_size, 3))*0.5
#         frame[0][height_black_band[1]:256,-left_right_black_size:] = \
#             np.ones((256 - height_black_band[1], left_right_black_size, 3))*0.5
#         frame[0][-bottom_black_size:,:] = np.ones((bottom_black_size, 256,3))*0.5

        tmp = cv2.resize(frame[0], (720,720))
        tmp[:,:,[0,2]] = tmp[:,:,[2,0]]
        cv2.imshow('Stitched',tmp)
        cv2.waitKey(1)
        counter += 1
cv2.destroyAllWindows()

In [ ]:
import getpass

import os

password = getpass.getpass()
command = 'sudo -S modprobe -r v4l2loopback && sudo modprobe v4l2loopback devices=1 video_nr=5 card_label="Virtual" exclusive_caps=1 max_buffers=2'

os.system('echo %s | %s' % (password, command))

In [ ]:
       
with pyvirtualcam.Camera(width=256, height=256, fps=30,device='/dev/video5') as cam:
    while(1):
        start = time.time()
        res, frame = cap.read()
        if not res:
            print('no frame')

        frame = pre_process_frame(frame) 
        frame[0][:height_black_band[0],:] = regular_video[counter%len(regular_video)][:height_black_band[0],:] #256 x 256
        
        frame[0][height_black_band[0]:height_black_band[1],:] = np.zeros((height_black_band[1] - height_black_band[0], 256, 3))
        ## shift chin 
        frame[0][height_black_band[1]:,left_right_black_size + lateral_shift:256 - left_right_black_size + lateral_shift,:] = \
            frame[0][height_black_band[1]:,left_right_black_size :256 - left_right_black_size,:]
        frame[0][height_black_band[1]:256,0:left_right_black_size] = np.zeros((256 - height_black_band[1], left_right_black_size, 3))
        frame[0][height_black_band[1]:256,-left_right_black_size:] = np.zeros((256 - height_black_band[1], left_right_black_size, 3))
        frame[0][-bottom_black_size:,:] = np.zeros((bottom_black_size, 256,3))
  
    
    
#         frame[0][height_black_band[0]:height_black_band[1],:] = np.ones((height_black_band[1] - height_black_band[0], 256, 3))
        
#         ## shift chin 
#         frame[0][height_black_band[1]:,left_right_black_size + lateral_shift:256 - left_right_black_size + lateral_shift,:] = \
#             frame[0][height_black_band[1]:,left_right_black_size :256 - left_right_black_size,:]
        
#         frame[0][height_black_band[1]:256,0:left_right_black_size] = np.ones((256 - height_black_band[1], left_right_black_size, 3))
#         frame[0][height_black_band[1]:256,-left_right_black_size:] = np.ones((256 - height_black_band[1], left_right_black_size, 3))
#         frame[0][-bottom_black_size:,:] = np.ones((bottom_black_size, 256,3))
        

        cam.send((frame[0]*255).astype(np.uint8))
        cam.sleep_until_next_frame()
        counter = (counter+1)%len(regular_video)


# # predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True, cpu=False)
# imageio.mimsave('res.mp4', [img_as_ubyte(frame) for frame in predictions], fps=25)